**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Activate Calico

`minikube start --network-plugin=cni --cni=calico`


## Alternativley Install Cilium in Minikube
https://docs.cilium.io/en/stable/gettingstarted/minikube/

**Warning** this might be highly version critical and needs testing if the minikube version is different from 1.3.1

Activate Berkeley Packet Filters

`minikube ssh -- sudo mount bpffs -t bpf /sys/fs/bpf`


**This might take some 6 minutes**

`kubectl create -f https://raw.githubusercontent.com/cilium/cilium/1.6.1/install/kubernetes/quick-install.yaml`


In [1]:
kubectl get pods -A 

NAMESPACE     NAME                                       READY   STATUS    RESTARTS   AGE
kube-system   calico-kube-controllers-55ffdb7658-6dnz6   1/1     Running   0          86s
kube-system   calico-node-k7vz4                          1/1     Running   0          86s
kube-system   coredns-74ff55c5b-lzz5q                    1/1     Running   0          86s
kube-system   etcd-minikube                              1/1     Running   0          93s
kube-system   kube-apiserver-minikube                    1/1     Running   0          93s
kube-system   kube-controller-manager-minikube           1/1     Running   0          93s
kube-system   kube-proxy-64jt7                           1/1     Running   0          86s
kube-system   kube-scheduler-minikube                    1/1     Running   0          93s
kube-system   storage-provisioner                        1/1     Running   0          96s


## Create a test

It will deploy a simple probe and echo server running with multiple replicas. The probe will only report readiness while it can successfully reach the echo server:

In [2]:
kubectl get pods

No resources found in default namespace.


The Apache example follows 
https://supergiant.io/blog/understanding-network-policies-in-kubernetes/
with the updated versions of the Cilium installation above

In [3]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: sleep
spec:
  replicas: 1
  selector:
    matchLabels:
      app: sleep
  template:
    metadata:
      labels:
        app: sleep
    spec:
      containers:
      - name: sleep
        image: pstauffer/curl
        command: ["/bin/sleep", "3650d"]
        imagePullPolicy: IfNotPresent
EOF

deployment.apps/sleep created


In [4]:
kubectl get pods,svc

NAME                        READY   STATUS              RESTARTS   AGE
pod/sleep-56bc8d768-qpvwt   0/1     ContainerCreating   0          4s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   3m10s


In [5]:
SLEEP_POD=$(kubectl get pods | awk '/sleep-/{print $1}')
echo $SLEEP_POD

sleep-56bc8d768-qpvwt


In [6]:
kubectl exec $SLEEP_POD -- curl -m 5 -s heise.de

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.heise.de/">here</a>.</p>
</body></html>


In [8]:
kubectl exec $SLEEP_POD -- curl -m 5 -s google.de

<HTML><HEAD><meta http-equiv="content-type" content="text/html;charset=utf-8">
<TITLE>301 Moved</TITLE></HEAD><BODY>
<H1>301 Moved</H1>
The document has moved
<A HREF="http://www.google.de/">here</A>.
</BODY></HTML>


In [9]:
dig heise.de


; <<>> DiG 9.16.1-Ubuntu <<>> heise.de
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 29627
;; flags: qr rd ra; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;heise.de.			IN	A

;; ANSWER SECTION:
heise.de.		5955	IN	A	193.99.144.80

;; Query time: 4 msec
;; SERVER: 192.168.49.1#53(192.168.49.1)
;; WHEN: Wed Jan 27 15:36:12 UTC 2021
;; MSG SIZE  rcvd: 53



## Networkpolicy


In [10]:
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: test-np
  namespace: default
spec:
  podSelector:
    matchLabels:
      app: sleep
  policyTypes:
  - Ingress
  - Egress
  ingress:
  - from:
    - namespaceSelector:
        matchLabels:
          project: dev
    - podSelector:
        matchLabels:
          role: frontend
    ports:
    - protocol: TCP
      port: 80
  egress:
  - to:
    - ipBlock:
        cidr: 193.99.144.80/32
    ports:
    - protocol: TCP
      port: 80
EOF

networkpolicy.networking.k8s.io/test-np created


In [21]:
kubectl exec $SLEEP_POD -- curl -m 5 -s heise.de
echo $?

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.heise.de/">here</a>.</p>
</body></html>
0


In [22]:
kubectl exec $SLEEP_POD -- curl -m 5  -s google.de
echo $?

command terminated with exit code 28
28


In [23]:
kubectl exec $SLEEP_POD -- curl -m 5 -s 193.99.144.80
echo $?

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.heise.de/">here</a>.</p>
</body></html>
0


# Include DNS

In [15]:
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: test-np
  namespace: default
spec:
  podSelector:
    matchLabels:
      app: sleep
  policyTypes:
  - Ingress
  - Egress
  ingress:
  - from:
    - namespaceSelector:
        matchLabels:
          project: dev
    - podSelector:
        matchLabels:
          role: frontend
    ports:
    - protocol: TCP
      port: 80
  egress:
  - to:
    - ipBlock:
        cidr: 193.99.144.80/32
    ports:
    - protocol: TCP
      port: 80
  - ports:
    - port: 53
      protocol: UDP
    - port: 53
      protocol: TCP
EOF

networkpolicy.networking.k8s.io/test-np configured


In [17]:
kubectl exec $SLEEP_POD -- curl -m 5 -s heise.de

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.heise.de/">here</a>.</p>
</body></html>


In [20]:
kubectl exec $SLEEP_POD -- curl -v -m 5 -s google.de

* Rebuilt URL to: google.de/
*   Trying 142.250.74.195...
* TCP_NODELAY set
*   Trying 2a00:1450:4001:821::2003...
* TCP_NODELAY set
* Immediate connect fail for 2a00:1450:4001:821::2003: Address not available
*   Trying 2a00:1450:4001:821::2003...
* TCP_NODELAY set
* Immediate connect fail for 2a00:1450:4001:821::2003: Address not available
*   Trying 2a00:1450:4001:821::2003...
* TCP_NODELAY set
* Immediate connect fail for 2a00:1450:4001:821::2003: Address not available
*   Trying 2a00:1450:4001:821::2003...
* TCP_NODELAY set
* Immediate connect fail for 2a00:1450:4001:821::2003: Address not available
*   Trying 2a00:1450:4001:821::2003...
* TCP_NODELAY set
* Immediate connect fail for 2a00:1450:4001:821::2003: Address not available
*   Trying 2a00:1450:4001:821::2003...
* TCP_NODELAY set
* Immediate connect fail for 2a00:1450:4001:821::2003: Address not available
* Connection timed out after 5000 milliseconds
* stopped the pause stream!
* Closing connection 0
command terminated wit

: 28

In [ ]:
kubectl get pods 